# Decision Tree Regression
<hr>
<br>

**`Standart Devation`**

<Br>
$Std = \sqrt{\dfrac{\sum{ (x-myu)^2}} {n}}$
<br>
    <br>
$myu = \dfrac{\sum{x}}{n}$
<br>
<hr>
<br>
    
**`Standart Devation Reduction`**
<br>
<br>
$SDTR = STD(Label) - STD(Feature, Label)$
    

In [10]:
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn import  datasets
import pandas as pd
import numpy as np

In [11]:
def std(arr):
    myu = arr.mean()
    return np.sqrt(np.sum(np.square(arr - myu))/ len(arr))

In [12]:
def stdr(arr1,arr2):
    std_arr = std(arr2)
    uniq = np.unique(arr1)
    list_stdr = list()
    for i in uniq:
        label = arr2[arr1==i]
        print(i,":", std(label), "\t",len(label))
        list_stdr.append([len(label), std(label)])
    list_stdr = np.array(list_stdr).T
    
    return std_arr - np.dot(list_stdr[0]/len(arr1), list_stdr[1])
    


In [13]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist/len(y)
    return(-np.sum([p * np.log2(p) for p in ps if p>0]))

In [14]:
# Node Class
class Node:
    
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf_node(self):
        return self.value is not None


# Decision Tree Regressor Class
class CustomDecisionTreeRegressor:
    def __init__(self, n_feats = None, max_depth = 100, min_samples_split = 2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None ,
                 random_state=None ,max_leaf_nodes=None, min_impurity_decrease=0.0, ccp_alpha=0.0):
        
        self.root = None
        self.n_feats = n_feats
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
    
    def fit(self, X, Y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        #self.col = list(X.columns)
        self.root = self.growTree(X, Y)

    def growTree(self, X, Y, depth = 0):
        
        df = pd.DataFrame(X.copy())
        df['y'] = Y
        
        ymean = np.mean(Y)
        
        self.mse = self.get_mse(Y, ymean)
        
        n_sample, n_feature = X.shape
        
        # stopping criteria
        if (depth >= self.max_depth or n_sample <= self.min_samples_split):
            leaf_value = np.mean(Y)
            return Node(value=leaf_value)

        feats_idxs = list(X.columns)

        best_feat, best_thresh = self.best_criteria(X, Y, feats_idxs)

        left_df, right_df = df[df[best_feat]<=best_thresh].copy(), df[df[best_feat]>best_thresh].copy()

        left = self.growTree(left_df.drop('y', axis=1), left_df['y'].values.tolist(), depth+1)
        right = self.growTree(right_df.drop('y', axis=1), right_df['y'].values.tolist(), depth+1)

        return Node(best_feat, best_thresh, left, right)
    
    
    # find out best criteria
    def best_criteria(self, X, Y, feats_idxs):
        
        df = X.copy()
        
        df['y'] = Y.copy()
        
        mse_base = self.mse
        
        best_feature = None
        best_thresh = None
        
        for feat in feats_idxs:
            
            xdf = df.sort_values(feat)
            
            x_mean = self.moving_average(xdf[feat], 2)

            for value in x_mean:
                left_y = xdf[xdf[feat] < value]['y'].values
                right_y = xdf[xdf[feat] >= value]['y'].values
                
                left_mean = 0
                right_mean = 0
                if len(left_y) > 0:
                    left_mean = np.mean(left_y)
                if len(right_y) > 0:
                    right_mean = np.mean(right_y)
                
                res_left = left_y - left_mean
                res_right = right_y - right_mean
                
                r = np.concatenate((res_left, res_right), axis=None)
                
                n = len(r)

                r = r ** 2
                r = np.sum(r)
                mse_split = r / n
                
                if mse_split < mse_base:
                    mse_base = mse_split
                    best_feature = feat
                    best_thresh = value
                    
        return (best_feature, best_thresh)
    
    def get_mse(self, y_true, y_hat):
        n = len(y_true)
        
        r = y_true - y_hat
        
        r = r ** 2
        
        r = np.sum(r)
        
        return r / n
    
    def moving_average(self, x:np.array, window : int):
        return np.convolve(x, np.ones(window), 'valid') / window 
    
    def predict(self, X):
        X = X.to_numpy().tolist()
        
        return np.array([self.traverse_tree(x, self.root) for x in X])

    def traverse_tree(self, x, node):
       
        if node.value is not None:
            return node.value
        
        fr = node.feature
        index = self.col.index(fr)

        if x[index] <= node.threshold:
            return self.traverse_tree(x, node.left)
        
        return self.traverse_tree(x, node.right)

In [15]:
def accuracy(y_true, y_pred):
    acc = np.sum(y_true==y_pred)/y_true
    return acc

In [16]:
data = datasets.load_diabetes()
print(data["DESCR"])
X = data["data"]
y = data["target"]

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [17]:
X

array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
         0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
        -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
         0.00286131, -0.02593034],
       ...,
       [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
        -0.04688253,  0.01549073],
       [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
         0.04452873, -0.02593034],
       [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
        -0.00422151,  0.00306441]])

In [18]:
y

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
        68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
        87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
       259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
       128.,  52.,  37., 170., 170.,  61., 144.,  52., 128.,  71., 163.,
       150.,  97., 160., 178.,  48., 270., 202., 111.,  85.,  42., 170.,
       200., 252., 113., 143.,  51.,  52., 210.,  65., 141.,  55., 134.,
        42., 111.,  98., 164.,  48.,  96.,  90., 162., 150., 279.,  92.,
        83., 128., 102., 302., 198.,  95.,  53., 134., 144., 232.,  81.,
       104.,  59., 246., 297., 258., 229., 275., 281., 179., 200., 200.,
       173., 180.,  84., 121., 161.,  99., 109., 115., 268., 274., 158.,
       107.,  83., 103., 272.,  85., 280., 336., 281., 118., 317., 235.,
        60., 174., 259., 178., 128.,  96., 126., 28

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
custom_tree = CustomDecisionTreeRegressor()

In [21]:
custom_tree.fit(X_train, y_train)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

# Decision Tree Classification
<hr>
<br>

**`Entropy`**

$E(S) = \sum_{i=1}^n (- p_i log_2 p_i)$

<hr>
<br>

**`Information Gain`**
<br>
$Gain(S,A) = E(S) - \sum_{(v Values(A))}  \dfrac{S}{S_v} E(S_v)  $

<hr>
<br>

**`Gini Index`**

$Gini = 1 - \sum_{i=1}^j p_i^2$


In [20]:
def compute_impurity(feature, criteria="entropy"):
    """
    Supported impurity criteria: "entropy", "gini"
    """
    probs = np.unique(feature, return_counts=True)[1] 
    probs = probs / len(feature)
    
    if criteria=="entropy":
        return -1 * np.sum(np.log2(probs) * probs )
    
    elif criteria=="gini":
        
        return 1 - np.sum(np.square(probs))
    
    else:
        raise ValueError("Unkown criteria")

In [150]:
def information_gain(data, target,  split_criterion="entropy"):
    entropy_list = list()
    weight_list = list()
    target_entropy = compute_impurity(target, split_criterion)   
    for i in np.unique(data):
        
        label = target[data == i]
        
        
        entropy = compute_impurity(feature=label,  criteria=split_criterion)
        
        
        entropy_list.append( entropy )
        weight_list.append(len(label)/len(data))
   
    information_gain = target_entropy - np.sum(np.array(entropy_list) * weight_list)
    return information_gain
        

In [151]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # Oxirgi chiqish qiymati uchun
        self.value = value

In [154]:
class CustomDecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=5):
        ''' Konstruktor '''
        
        # Bosh tugunni istalizatsiya qilish
        self.root = None
        
        # Bo'linish chegarasi bilan maksimal chuqurlik
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' Daraxtni qurish uchun rekursiv funksiya''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # To'tatish shartlari bajarilmaguncha bo'linadi
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            
            # Eng yaxshi bo'linishni topish funksiyasi
            best_split = self.get_best_split(dataset, num_samples, num_features)
            
            # Info gain ni musbatlikka tekshiryapmiz
            if best_split["info_gain"]>0:
                
                # chapga bo'linish
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                
                # onga bo'linish
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                
                # Daraxt tugunlarini qaytaradi
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        
        # Chiquvchi qiymatnin hisoblash funksiyasi
        leaf_value = self.calculate_leaf_value(Y)
        
        # Chiquvchi qiymatni qaytaradi
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' Eng yaxshi bo'linishni topish funksiyasi '''
        
        # Bo'linishni saqlash uchun fuksiya
        best_split = {}
        max_info_gain = -float("inf")
        
        # Barcha feature ni tekshirish
        for feature_index in range(num_features):
            
            feature_values = dataset[:, feature_index]
            
            thresh_min, thresh_max = np.min(np.unique(feature_values)), np.max(np.unique(feature_values))
            possible_thresholds = np.arange(thresh_min, thresh_max+0.1, 0.1)
            
            
            # Eng yaxshi bo'linishni topish            
            for threshold in possible_thresholds:
                
                # Chegara bo'yicha bo'linishlarni olish                
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                
                # Bo'lingan datalarni tekshirish
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    
                    # Information Gain ni hisoblash
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    
                    # Eng yaxshi bo'linishni saqlash
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # Eng yaxshi bo'linishni saqlash
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' Ma'lumotlarni bo'lish funksiyasi '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' Information Gain ni hisoblash funksiyasi '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' Entropy ni hisoblash funksiyasi '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' Gini indexni hisoblash fuksiyasi '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)